In [1]:
import os

from dateutil.parser import isoparse
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

from insight.catalog import InsightCatalog
from insight.rotate import rotate_zne
from insight.util import EventDownloader, TraceFile

%load_ext autoreload
%autoreload 1

# Parameters

In [2]:
DATA_DIR = "../data/"
DATALESS_SEED = "ELYSE.dataless"
CATALOG_BASE_URL = "https://www.seis-insight.eu/static/mqs-catalogs/v14/"
CATALOG_STD = "events_extended_multiorigin_v14_2023-01-01.xml"
CATALOG_EXT = "events_mars_extended_multiorigin_v14_2023-01-01.xml"

# https://www.iris.edu/hq/sis/insight
network = "XB"
station = "ELYSE"
location = "02"
min_ang, max_ang = 30, 90
before, after = 10, 120
inv_time = UTCDateTime(2020, 1, 1)  # Only used to get channel metadata
channels = ["BHU", "BHV", "BHW"]

# # RFs
# f0 = 2.0
# itmax = 400
# minderr = 0.001

## Fetch Inventory
Get lat/lon and sensor orientations from Iris

In [3]:
client = Client("IRIS")
insight = client.get_stations(network=network, station=station, location=location, channel="BH?", level="channel")[0]
lat, lon = insight[0].latitude, insight[0].longitude
bh_orientations = {
    ch: insight.get_orientation(f"{network}.{station}.{location}.{ch}", inv_time)
    for ch in channels
}

## Build Catalog

In [4]:
cb = InsightCatalog(CATALOG_BASE_URL + CATALOG_EXT, DATA_DIR)
catalog = cb.parse()
events = catalog.events.set_index("earthquake name")
events

,id,region name,time,quality,mqs_azimuth,mqs_distance,est_p_arrival,M_w
earthquake name,,,,,,,,
S1415a,mqs2022wrzi,Elysium Southwest,2022-11-19T21:53:34.0000Z,B,108.8539635,4.951151082e-05,2022-11-19T21:55:49.119346Z,3.3
S1337a,mqs2022rbsc,Elysium Southwest,2022-08-31T20:23:46.685686Z,B,108.8539635,4.951151082e-05,2022-08-31T20:28:46.685686Z,3.3
S1237b,mqs2022jvts,Elysium Southwest,2022-05-20T23:37:51.098952Z,D,108.8539635,4.951151082e-05,2022-05-20T23:40:51.098952Z,NaN
S1237a,mqs2022jvst,Elysium Southwest,2022-05-20T23:08:41.115997Z,C,108.8539635,4.951151082e-05,2022-05-20T23:13:41.115997Z,2.6
S1235a,mqs2022jrva,Elysium Southwest,2022-05-18T19:45:44.84616Z,C,108.8539635,4.951151082e-05,2022-05-18T19:50:44.84616Z,2.7
...,...,...,...,...,...,...,...,...
S0128a,mqs2019gudd,Elysium Southwest,2019-04-07T09:31:52.0000Z,B,108.8539635,4.951151082e-05,2019-04-07T09:33:36.50275Z,2.2
S0116a,mqs2019fxyu,Elysium Southwest,2019-03-26T06:22:19.90061Z,D,108.8539635,4.951151082e-05,2019-03-26T06:27:19.90061Z,NaN
S0105a,mqs2019fddj,Aeolis Northeast,2019-03-14T20:59:21.050826Z,B,112,32.52202202,2019-03-14T21:03:28.123118Z,2.9


## Fetch 'A' Quality Events

In [5]:
st_uvw = {}

events_dl = EventDownloader(DATA_DIR, before=before, after=after)
event_ids = events[events["quality"] == "A"].index

for evt in event_ids:
    e = events.loc[evt]
    st_uvw[evt] = events_dl.get_stream(
        network=network,
        station=station,
        location=location,
        channel="BH?",
        est_p_arrival=isoparse(e.est_p_arrival),
        evt_id=e.id
    )

## Rotate to ZNE

In [6]:
st_zne = {}

for evt in event_ids:
    st_zne[evt] = rotate_zne(st_uvw[evt], bh_orientations)

## Save rotated streams and catalog to disk

Sort by M_w

In [13]:
events.loc[event_ids].sort_values("M_w", ascending=False).to_parquet(os.path.join(DATA_DIR, "a_quality.parquet"), index=True)
zne_dst_dir = os.path.join(DATA_DIR, "zne")
if not os.path.exists(zne_dst_dir):
    os.mkdir(zne_dst_dir)

for evt in event_ids:
    trace_file = TraceFile(network, station, location, events.loc[evt].id, format="MSEED")
    st_zne[evt].write(os.path.join(zne_dst_dir, str(trace_file)), format="MSEED")

/Users/tom/Library/Caches/pypoetry/virtualenvs/insight-notebooks-clKRl_iU-py3.11/lib/python3.11/site-packages/obspy/io/mseed/core.py:770: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


In [12]:
st_zne["S1222a"][0].stats

         network: XB
         station: ELYSE
        location: 02
         channel: BHZ
       starttime: 2022-05-04T23:27:35.204000Z
         endtime: 2022-05-04T23:29:45.154000Z
   sampling_rate: 20.0
           delta: 0.05
            npts: 2600
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'M', 'number_of_records': 24, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 36864})